In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Video 4  Multiple Negatives Ranking Loss

In [2]:
import datasets 

snli = datasets.load_dataset('snli', split='train')
mnli = datasets.load_dataset('glue', 'mnli', split='train')
mnli = mnli.remove_columns(['idx'])
snli = snli.cast(mnli.features)

dataset = datasets.concatenate_datasets([snli, mnli])

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


...

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Casting the dataset:   0%|          | 0/56 [00:00<?, ?ba/s]

In [3]:
print(f'Before: {len(dataset)} rows')
dataset = dataset.filter(
    lambda x: True if x['label'] == 0 else False 
)
print(f'After: {len(dataset)} rows')

Before: 942854 rows


  0%|          | 0/943 [00:00<?, ?ba/s]

After: 314315 rows


In [4]:
from transformers import BertTokenizer 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = dataset.map(
    lambda x: tokenizer(x['premise'], max_length=128, padding='max_length', truncation=True), batched=True
)

dataset = dataset.rename_column('input_ids', 'anchor_ids')
dataset = dataset.rename_column('attention_mask', 'anchor_mask')

dataset

  0%|          | 0/315 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label', 'anchor_ids', 'token_type_ids', 'anchor_mask'],
    num_rows: 314315
})

In [5]:
dataset = dataset.map(
    lambda x: tokenizer(x['hypothesis'], max_length=128, padding='max_length', truncation=True), batched=True
)

dataset = dataset.rename_column('input_ids', 'positive_ids')
dataset = dataset.rename_column('attention_mask', 'positive_mask')

dataset = dataset.remove_columns(['premise', 'hypothesis', 'label', 'token_type_ids'])
dataset

  0%|          | 0/315 [00:00<?, ?ba/s]

Dataset({
    features: ['anchor_ids', 'anchor_mask', 'positive_ids', 'positive_mask'],
    num_rows: 314315
})

In [6]:
dataset.set_format(type='torch', output_all_columns=True)

In [7]:
import torch 

batch_size = 32 

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

![](https://d33wubrfki0l68.cloudfront.net/03eab5d2ff299fa356d276f145360508faf7e1e0/f5708/images/fine-tuning-sentence-transformers-mnr-loss-2.jpg)

In [8]:
from transformers import BertModel 

model = BertModel.from_pretrained('bert-base-uncased')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def mean_pool(token_embeds, attention_mask):
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float() 
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(in_mask.sum(1), min=1e-9)
    return pool

In [10]:
cos_sim = torch.nn.CosineSimilarity() 

In [11]:
a = torch.randn((32, 768))
p = torch.randn((32, 768))

In [12]:
a.shape, p.shape

(torch.Size([32, 768]), torch.Size([32, 768]))

In [13]:
scores = [] 
for a_i in a:
#     print(a_i.shape)
    scores.append(cos_sim(a_i.reshape(1, a_i.shape[0]), p))
#     print(a_i.reshape(1, a_i.shape[0]).shape)

In [14]:
scores = torch.stack(scores)
scores

tensor([[ 0.0258, -0.0194, -0.0094,  ..., -0.0721, -0.0005, -0.0094],
        [-0.0090, -0.0113,  0.0246,  ..., -0.0115, -0.0077,  0.0295],
        [ 0.0381,  0.0102,  0.0553,  ...,  0.0168, -0.0144, -0.0530],
        ...,
        [-0.1051, -0.0323,  0.0312,  ...,  0.0350, -0.0179,  0.0571],
        [ 0.0057, -0.0727, -0.0531,  ..., -0.0394,  0.0700,  0.0325],
        [-0.0101,  0.0294,  0.0086,  ..., -0.0053, -0.0201, -0.0523]])

In [15]:
scores.shape

torch.Size([32, 32])

In [16]:
labels = torch.tensor(range(len(scores)), dtype=torch.long, device=scores.device)
labels

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [17]:
loss_func = torch.nn.CrossEntropyLoss()

In [18]:
loss_func(scores, labels)

tensor(3.4680)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [20]:
scale = 20.0 
cos_sim.to(device)
loss_func.to(device)

CrossEntropyLoss()

In [21]:
from transformers.optimization import get_linear_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr=2e-5)
total_steps = int(len(dataset) / batch_size)
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optim, num_warmup_steps=warmup_steps,
    num_training_steps=total_steps - warmup_steps
)
epochs = 1

In [22]:
from tqdm import tqdm 

for epoch in range(epochs):
    model.train() 
    loop = tqdm(loader, leave=True)
    for batch in loop:
        optim.zero_grad() 
        anchor_ids = batch['anchor_ids'].to(device)
        anchor_mask = batch['anchor_mask'].to(device)
        pos_ids = batch['positive_ids'].to(device)
        pos_mask = batch['positive_mask'].to(device)
        
        a = model(anchor_ids, attention_mask=anchor_mask)[0] 
        p = model(pos_ids, attention_mask=pos_mask)[0] 
        a = mean_pool(a, anchor_mask)
        p = mean_pool(p, pos_mask)
        
        scores = torch.stack([cos_sim(a_i.reshape(1, a_i.shape[0]), p) for a_i in a])
        labels = torch.tensor(range(len(scores)), dtype=torch.long, device=scores.device)
        loss = loss_func(scores * scale, labels)
        loss.backward() 
        optim.step() 
        scheduler.step() 
        
        loop.set_description(f'Epoch {epoch + 1}')
        loop.set_postfix(loss=loss.item())

Epoch 1:  12%|█▏        | 1224/9823 [24:43<2:53:39,  1.21s/it, loss=1.45] 


KeyboardInterrupt: 

In [23]:
import os 

model_path = 'model1'

if not os.path.exists(model_path):
    os.mkdir(model_path)

model.save_pretrained(model_path)

In [24]:
snli = datasets.load_dataset('snli', split='train')
mnli = datasets.load_dataset('glue', 'mnli', split='train')
mnli = mnli.remove_columns(['idx'])
snli = snli.cast(mnli.features)

dataset = datasets.concatenate_datasets([snli, mnli])

Casting the dataset:   0%|          | 0/56 [00:00<?, ?ba/s]

In [25]:
print(f'Before: {len(dataset)} rows')
dataset = dataset.filter(
    lambda x: True if x['label'] == 0 else False 
)
print(f'After: {len(dataset)} rows')

Before: 942854 rows


  0%|          | 0/943 [00:00<?, ?ba/s]

After: 314315 rows


In [26]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 306.0 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=dac67ab71f7d1fef421187a22395a356d0e9c7137037930241a5f34b196fa645
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [27]:
from sentence_transformers import InputExample 

train_samples = [] 

for row in tqdm(dataset):
    train_samples.append(InputExample(texts=[row['premise'], row['hypothesis']]))

100%|██████████| 314315/314315 [00:24<00:00, 12889.64it/s]


In [28]:
from sentence_transformers import datasets 

batch_size = 32 

loader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=batch_size)

In [29]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('bert-base-uncased')
pooler = models.Pooling(bert.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

model = SentenceTransformer(modules=[bert, pooler])

model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [30]:
from sentence_transformers import losses 

loss = losses.MultipleNegativesRankingLoss(model)

In [31]:
epochs = 1 
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='model2',
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9822 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
import datasets

sts = datasets.load_dataset('glue', 'stsb', split='validation')
sts

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 1500
})

In [33]:
sts = sts.map(lambda x: {'label': x['label'] / 5.0})

  0%|          | 0/1500 [00:00<?, ?ex/s]

In [34]:
from sentence_transformers import InputExample

samples = []
for sample in sts:
    samples.append(InputExample(
        texts=[sample['sentence1'], sample['sentence2']],
        label=sample['label']
    ))

In [35]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    samples, write_csv=False
)

In [36]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('model2')

evaluator(model)

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-65275058-3aabb36c5b58333c7c1e1bf8)

Repository Not Found for url: https://huggingface.co/api/models/sentence-transformers/model2.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.